In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV



In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")



In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
target = train_df["target"]
train_df = train_df.drop(["target"],axis=1)

In [ ]:
#looks like all are correlated well with each other
plt.figure(figsize=(15,15))
sns.heatmap(train_df[2:].corr())
plt.show()

In [ ]:
param_grid = {'C'     : [0.001,0.01,0.1,1,10,100],
              'gamma' : [0.001,0.01,0.1,1,10,100]}

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit


shuffle_split = StratifiedShuffleSplit(test_size=0.8,train_size=0.2,n_splits=20)

In [ ]:
grid_search = GridSearchCV(estimator = SVC(probability=True),param_grid = param_grid, cv = shuffle_split)

In [ ]:
grid_search.fit(train_df,target)

In [ ]:
print("Best parameters : {}".format(grid_search.best_params_))
print("Best cross validation score: {:.2f}".format(grid_search.best_score_))
print("Best estimator: {}".format(grid_search.best_estimator_))


In [ ]:
results = pd.DataFrame(grid_search.cv_results_)

In [ ]:
# As we are searching for a two dimensional grid of parameters (C and Gamma), this is best
# visualized in heat map. 
scores = np.array(results.mean_test_score).reshape(6,6) #np.array flattens the col, then we reshape to df.
sns.heatmap(scores,xticklabels=param_grid['gamma'],
             yticklabels=param_grid['C'],cmap='viridis', annot= True, linewidth=0.5)

In [ ]:
#print( "Predictions on test set {}".format(grid_search.predict(test_df)))
prediction = grid_search.predict_proba(test_df)[:,1]

In [ ]:
prediction[:5]

In [ ]:
#print test file 
sub_df = pd.DataFrame({"id":test_df["id"].values})
sub_df["target"] = prediction
sub_df.to_csv("baseline_svc.csv", index=False)